In [ ]:
# Imports

from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error
import os

In [ ]:
# Function to create a connection to the database

def create_connection(db_file, delete_db=False):
    
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

In [ ]:
# Function to create a database

def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
# Execute SQL statement

def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [ ]:
# Read the Non-Normalized Data

df=pd.read_csv("GlobalLandTemperaturesByCity.csv")
data=df[df.dt>"1949-12-31"]
    

In [ ]:
# Extract cities and their latitude and longitude

data_for_loc_table=data[['City','Latitude','Longitude']]
data_for_loc_table=data_for_loc_table.to_records(index=False)
result_data_for_loc_table=list(data_for_loc_table)

In [ ]:
# Extract Countries

records=data.to_records(index=False)
result=list(records)
req_resut=[]
for i in result:
    req_resut.append((i[4],))


In [ ]:
# Convert the list of cities into a list with unique names

req_resut=list((set(req_resut)))

In [ ]:
# Create a connection to the database
conn = create_connection('database.db')
cur=conn.cursor()

In [ ]:
# Drop City Table and the Country Table if they exist and create a new Country Table and Insert the data

execute_sql_statement("DROP TABLE IF EXISTS City_Table",conn)
execute_sql_statement("DROP TABLE IF EXISTS Country_Table",conn)
create_table(conn,'''CREATE TABLE Country_Table (PK INTEGER  PRIMARY KEY, Country_Name VARCHAR)''')
cur.executemany("INSERT INTO Country_Table ('Country_Name') Values (?)",req_resut)

In [ ]:
# Make a dictionary of the country data and their IDs
abc=execute_sql_statement("Select * from Country_Table", conn)
abc1={}
for i in abc:
    abc1[i[1]]=i[0]

In [ ]:
req_result_2=[]
for i in result:
    req_result_2.append((i[3],abc1[i[4]]))

In [ ]:
# Create the City Table
create_table(conn,'''CREATE TABLE City_Table (City_Id INTEGER  PRIMARY KEY, City_Name VARCHAR , Country_ID INTEGER,
FOREIGN KEY(Country_ID) REFERENCES Country_Table(PK))''')

In [ ]:
# Extract the data for City Table
city=[]
for i in result:
   # j=j+1
    city.append((i[3],abc1[i[4]]))
city_country_unique=list((set(city)))

In [ ]:
# Insert the data for City Table
cur.executemany("INSERT INTO City_Table ('City_Name','Country_ID') Values (?,?)",city_country_unique)

In [ ]:
#df=execute_sql_statement("Select * from City_Table", conn)
df=pd.read_sql_query("Select * from City_Table", conn)
display(df)

In [ ]:
# Build a dictionary of the city data and their IDs
city_index=execute_sql_statement("Select * from City_Table", conn)
city_indexes={}
for i in abc:
    abc1[i[1]]=i[0]

In [ ]:
# Displaying the country data
df1=pd.read_sql_query("Select * from Country_Table", conn)
display(df1)

In [ ]:
# Join the data from the City Table and the Country Table and display the data

execute_sql_statement("select * from City_Table join Country_Table on Country_Table.PK = City_Table.Country_ID AND Country_Table.Country_Name='India' LIMIT 10",conn)

In [ ]:
# City IDs Dictionary

city_table_values=execute_sql_statement("select * from City_Table ",conn)
city_ids={}
for i in city_table_values:
    city_ids[i[1]]=i[0]
city_ids
    

In [ ]:
# Create the Temperature Table

create_table(conn,'''CREATE TABLE Temperature (PK INTEGER  PRIMARY KEY , Date VARCHAR, Avg_Temperature VARCHAR, City_ID INTEGER,
FOREIGN KEY(City_ID) REFERENCES City_Table(City_Id))''')

In [ ]:
# Allocate the data to the Temperature Table and their corresponding City IDs

temp_table_values=[]
j=1
for i in result:
    j=j+1
    temp_table_values.append((i[0],i[1],city_ids[i[3]]))

In [ ]:
# Insert the data into the Temperature Table

cur.executemany("INSERT INTO Temperature ('Date','Avg_Temperature','City_ID') Values (?,?,?)",temp_table_values)

In [ ]:
# Displaying the Temperature Table

df1=pd.read_sql_query("Select * from Temperature", conn)
display(df1)

In [ ]:
# Create the Location Table

create_table(conn,'''CREATE TABLE Loc_Table (ID INTEGER  PRIMARY KEY , City_ID INTEGER, LAT FLOAT,LONG FLOAT,
FOREIGN KEY(City_ID) REFERENCES City_Table(City_ID))''')

In [ ]:
# Extract the data for the Location Table, Database consists of N,W,E,S notation, we are going to convert them to the decimal notation

#Lat= North is positive, South is negative
#Long= West is negative, East is positive
data_to_check_loc=result_data_for_loc_table
data_to_input_loc_table=[]
j=0
for i in data_to_check_loc:
    t=[0,0,0]
    if i[1][-1]=='N':
         t[1]=(float(i[1][:-1]))
    if i[1][-1]=='S':
         t[1]=float(i[1][:-1])*(-1)
    if i[2][-1]=='E':
         t[2]=float(i[2][:-1])  
    if i[2][-1]=='W':
         t[2]=float(i[2][:-1])*(-1)
    t[0]=i[0]
    tuple1=(city_ids[t[0]],t[1],t[2])
    data_to_input_loc_table.append(tuple1)
data_to_input_loc_table

In [ ]:
# Insert the data into the Location Table

cur.executemany("INSERT INTO Loc_Table ('City_ID','LAT','LONG') Values (?,?,?)",data_to_input_loc_table)

In [ ]:
# Displaying the Location Table

df1=pd.read_sql_query("Select * from Loc_Table ", conn)
display(df1)

In [ ]:
# df1=pd.read_sql_query("select * from Loc_Table join Temperature on Loc_Table.City_ID=Temperature.City_ID join city_table on city_table.city_id=Temperature.City_ID join country_table on country_table.pk=city_table.country_id group by Date LIMIT 10", conn)
# display(df1)
#("select * from Loc_Table join Temperature on Loc_Table.City_ID=Temperature.City_ID join city_table on city_table.city_id=Temperature.City_ID join country_table on country_table.pk=city_table.country_id LIMIT 10",conn)

In [ ]:
# Commit the changes

conn.commit()